In this excerise we will try to predict if a customer will subscribe to a term deposit , when is called from a call center.
The details of the dataset can be found here. [Portugese bank marketing dataset](https://archive.ics.uci.edu/ml/datasets/Bank+Marketing)

Attribute Information:

Input variables:
#### Bank client data:
1 - age (numeric)

2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')

3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)

4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')

5 - default: has credit in default? (categorical: 'no','yes','unknown')

6 - housing: has housing loan? (categorical: 'no','yes','unknown')

7 - loan: has personal loan? (categorical: 'no','yes','unknown')

#### Related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')

9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')

10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')

11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.

#### Other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)

13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)

14 - previous: number of contacts performed before this campaign and for this client (numeric)

15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')

#### Social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)

17 - cons.price.idx: consumer price index - monthly indicator (numeric)

18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)

19 - euribor3m: euribor 3 month rate - daily indicator (numeric)

20 - nr.employed: number of employees - quarterly indicator (numeric)


Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import  RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
plt.rc("font", size=14)

### 1. Read the dataset & drop columns with NA values.


In [2]:
data = pd.read_csv('data/org_banking.txt', header=0)
data = data.dropna()
print(data.shape)
print(list(data.columns))

(41188, 21)
['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp_var_rate', 'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed', 'y']


### 2. Data conversion.
We will convert basic education of different types into one type.This should reduce the cardinality of `education` feature.

In [3]:
data['education']=np.where(data['education'] =='basic.9y', 'basic', data['education'])
data['education']=np.where(data['education'] =='basic.6y', 'basic', data['education'])
data['education']=np.where(data['education'] =='basic.4y', 'basic', data['education'])

In [4]:
data['education'].unique()

array(['basic', 'unknown', 'university.degree', 'high.school',
       'professional.course', 'illiterate'], dtype=object)

In [5]:
count_no_sub = len(data[data['y']==0])
count_sub = len(data[data['y']==1])
pct_of_no_sub = count_no_sub/(count_no_sub+count_sub)
print("Percentage of no subscription is", pct_of_no_sub*100)
pct_of_sub = count_sub/(count_no_sub+count_sub)
print("Percentage of subscription", pct_of_sub*100)

Percentage of no subscription is 88.73458288821988
Percentage of subscription 11.265417111780131


### 3. Quick preview of the data.
The data that we have indeed has a lot of string values and we will have to convert them to numreical values.
This can be done using Onehotencoding.


In [6]:
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,44,blue-collar,married,basic,unknown,yes,no,cellular,aug,thu,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1,0
1,53,technician,married,unknown,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-0.1,93.200,-42.0,4.021,5195.8,0
2,28,management,single,university.degree,no,yes,no,cellular,jun,thu,...,3,6,2,success,-1.7,94.055,-39.8,0.729,4991.6,1
3,39,services,married,high.school,no,no,no,cellular,apr,fri,...,2,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,0
4,55,retired,married,basic,no,yes,no,cellular,aug,fri,...,1,3,1,success,-2.9,92.201,-31.4,0.869,5076.2,1


#### 4. Remove co-linear features
It is always a good idea to remove the co-linear feautures. The following code snippet removes co-linear features.

In [7]:
# Create correlation matrix
corr_matrix = data.corr().abs()
# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
# Drop features 
data.drop(data[to_drop], axis=1)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,nr_employed,y
0,44,blue-collar,married,basic,unknown,yes,no,cellular,aug,thu,210,1,999,0,nonexistent,1.4,93.444,-36.1,5228.1,0
1,53,technician,married,unknown,no,no,no,cellular,nov,fri,138,1,999,0,nonexistent,-0.1,93.200,-42.0,5195.8,0
2,28,management,single,university.degree,no,yes,no,cellular,jun,thu,339,3,6,2,success,-1.7,94.055,-39.8,4991.6,1
3,39,services,married,high.school,no,no,no,cellular,apr,fri,185,2,999,0,nonexistent,-1.8,93.075,-47.1,5099.1,0
4,55,retired,married,basic,no,yes,no,cellular,aug,fri,137,1,3,1,success,-2.9,92.201,-31.4,5076.2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,59,retired,married,high.school,unknown,no,yes,telephone,jun,thu,222,1,999,0,nonexistent,1.4,94.465,-41.8,5228.1,0
41184,31,housemaid,married,basic,unknown,no,no,telephone,may,thu,196,2,999,0,nonexistent,1.1,93.994,-36.4,5191.0,0
41185,42,admin.,single,university.degree,unknown,yes,yes,telephone,may,wed,62,3,999,0,nonexistent,1.1,93.994,-36.4,5191.0,0
41186,48,technician,married,professional.course,no,no,yes,telephone,oct,tue,200,2,999,0,nonexistent,-3.4,92.431,-26.9,5017.5,0


### 5. Split the data in test and train.

In [8]:
y = data.y.values
X_df = data.drop(columns=['y'],axis=1)

#### Label Encodings
There are two popular ways to do this: label encoding and one hot encoding.

For label encoding, a different number is assigned to each unique value in the feature column. A potential issue with this method would be the assumption that the label sizes represent ordinality (i.e. a label of 3 is greater than a label of 1).

For one hot encoding, a new feature column is created for each unique value in the feature column. The value would be 1 if the value was present for that observation and 0 otherwise. 
This method however could easily lead to an explosion in number of features and lead to the curse of dimensionality.

In [9]:
# Label encode categorical variables.
label_encoder = LabelEncoder()
mappings = []

# Desired label orders for categorical columns.
educ_order = ['unknown', 'illiterate','basic', 'high.school', 'professional.course', 'university.degree']
month_order = ['mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
day_order = ['mon', 'tue', 'wed', 'thu', 'fri']

# using cat.codes for order, one hot for high cardinality and weak case of cardinality.
def ordered_labels(df, col, order):
    df[col] = df[col].astype('category')
    df[col] = df[col].cat.reorder_categories(order, ordered=True)
    df[col] = df[col].cat.codes.astype(int)

# Use dummy variables for occupation
X_df = pd.concat([X_df, pd.get_dummies(X_df['job'])],axis=1).drop('job',axis=1)

# Use ordered cat.codes for days, months, and education
ordered_labels(X_df, 'education', educ_order)
ordered_labels(X_df, 'month', month_order)
ordered_labels(X_df, 'day_of_week', day_order)

# Same label encoding for rest since low cardinality
for i, col in enumerate(X_df):
    #If the data is of type string,apply LabelEncoding
    if X_df[col].dtype == 'object':
        X_df[col] = label_encoder.fit_transform(np.array(X_df[col].astype(str)).reshape((-1,)))
        mappings.append(dict(zip(label_encoder.classes_, range(1, len(label_encoder.classes_)+1))))

X_train, X_test, y_train, y_test = train_test_split(X_df,y,test_size=0.3)

### 6. Train a Logistic Regression.

In [10]:
#Train and Logistic Regression:
from sklearn.metrics import plot_roc_curve,roc_auc_score
from sklearn.preprocessing import StandardScaler,MaxAbsScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

lr_scaler = StandardScaler()
lr_clf = LogisticRegression(max_iter=2500)

pipeline = Pipeline([('scaler',lr_scaler),('classifier',lr_clf)])
pipeline.fit(X_train, y_train)

print(f'Logistic regression train AUC  :{roc_auc_score(y_train,pipeline.predict_proba(X_train)[:,1])}')
print(f'Logistic regression test AUC  :{roc_auc_score(y_test,pipeline.predict_proba(X_test)[:,1])}')

Logistic regression train AUC  :0.9306091188378088
Logistic regression test AUC  :0.9242329302376576


#### 6.1 How much does the results vary ?

In [13]:
from sklearn.model_selection import cross_validate
res = cross_validate(pipeline,X_train,y_train,cv=5,return_train_score=True)
print(pd.DataFrame(res))

   fit_time  score_time  test_score  train_score
0  0.124947    0.003481    0.910179     0.911507
1  0.099754    0.003438    0.911550     0.911814
2  0.103298    0.003342    0.911030     0.911945
3  0.098676    0.003447    0.911030     0.911641
4  0.094948    0.003326    0.912071     0.910731


### 7. Tune a Logisitic Regression Model

In [11]:
# Create Stratified KFold cross validation object :
from sklearn.model_selection import StratifiedKFold
sfk = StratifiedKFold(n_splits=5)
# Create param grid.
param_grid = [
    {
    'classifier__penalty' : ['l1', 'l2'],
    'classifier__C' : np.arange(1,10,0.5),
    'classifier__solver' : ['liblinear'] },
]

# Create grid search object
clf = GridSearchCV(pipeline, param_grid = param_grid, cv = sfk, verbose=True, n_jobs=-1)
# Take the best estimator and fit on entire training data
best_clf = clf.fit(X_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   18.1s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  1.2min finished


In [12]:
print(f'Logistic regression train AUC  :{roc_auc_score(y_train,best_clf.best_estimator_.predict_proba(X_train)[:,1])}')
print(f'Logistic regression test AUC  :{roc_auc_score(y_test,best_clf.best_estimator_.predict_proba(X_test)[:,1])}')

Logistic regression train AUC  :0.9305973653632285
Logistic regression test AUC  :0.9242144154308856


### 8. Train a RandomForestClassifier Model

In [15]:
# Create a RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=10,random_state=73)
# Creae a pipeline.
rf_pipeline = Pipeline([('classifier',lr_clf)])
rf_pipeline.fit(X_train, y_train)

print(f'RF regression train AUC  :{roc_auc_score(y_train,rf_pipeline.predict_proba(X_train)[:,1])}')
print(f'RF regression test AUC  :{roc_auc_score(y_test,rf_pipeline.predict_proba(X_test)[:,1])}')

RF regression train AUC  :0.9283415746520076
RF regression test AUC  :0.9230833649929941


#### How much do the results vary ?

In [16]:
rf_res = cross_validate(rf_pipeline,X_train,y_train,cv=5,return_train_score=True)
print(pd.DataFrame(rf_res))

   fit_time  score_time  test_score  train_score
0  2.637923    0.002243    0.909312     0.912591
1  3.277763    0.002240    0.911724     0.912031
2  1.383972    0.002260    0.913285     0.911901
3  0.468874    0.002216    0.910510     0.911771
4  0.373093    0.002220    0.912071     0.911554


### Excersise
1. Tune a RF model. Does it perform better or worse than the Linear Model ?

In [17]:
pipe = Pipeline([('classifier',RandomForestClassifier())])

# Create param grid.

# Create grid search object

# Fit on data

# Calculate ROC AUC score on Training and Test data .

2. Calculate the cross validation score for the Gridsearched Logistic Regression and RandomForest models.

* Which model overfits more ? Overfitting is defined as the absolute difference between training and test score.
* Which model will you suggest to use and why ?